In [289]:
driver = webdriver.Chrome()

driver.get(url)
driver.implicitly_wait(3)

html_page = driver.page_source
driver.quit()

soup = BeautifulSoup(html_page, 'lxml')
reviews = soup.findAll('div', attrs={"data-hook": "review"})

rev = []
for review in reviews:
    name = review.find('span', class_='a-profile-name').text.strip()    

    # Amazon's format is "x.0 stars out of 5" where x = # of stars
    rating = review.find('span', class_='a-icon-alt').text.strip()[0]

    review = review.find('span', attrs={'data-hook': 'review-body'}).text.strip()

    rev.append({'customer_name': name, 'rating': rating,'review': review})

# for customer name, find nested span class="a-profile-name"
# for customer rating, find nested span class="a-icon-alt"
# for customer review, find nested span data-hook="review"

In [293]:
for review in rev:
    print(review)

{'customer_name': 'Hayley', 'rating': '1', 'review': 'To soft'}
{'customer_name': 'Rohit', 'rating': '1', 'review': 'I had bought a refurbished iPhone 14 that was unlocked and was having troubles setting up the eSIM (because iphone14 and up are eSIM only) through various trials and errors we had looked up the IEMI number up on T-Mobile data base and it had been reported as stolen through xfinity, now I have a phone that I can’t uses because of this, unbelievable.'}
{'customer_name': 'Krystal', 'rating': '1', 'review': "This was the worst decision. I got the phone and the screen did not work. The screen is not apple and can tell it doesn't correctly fit. Even if I replaced it, T-Mobile said I would have the same problem. Don't buy from 3rd party. This was a scam."}
{'customer_name': 'Zhangbaoquan', 'rating': '1', 'review': 'The screen often goes black and the phone freezes. This phone must have been severely dropped or soaked in water. It is unethical to sell such a phone.'}
{'customer_

In [256]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [325]:
def __get_rated_reviews(url: str):
    driver = webdriver.Chrome()

    # make browser headless so it works in the background
    options = webdriver.ChromeOptions()
    options.add_argument("headless")

    driver.get(url)
    driver.implicitly_wait(3)

    html_page = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html_page, 'lxml')
    html_reviews = soup.findAll('div', attrs={"data-hook": "review"})

    reviews = []
    for html_review in html_reviews:
        # extract text from various span tags and clean up newlines in their strings
        name = html_review.find('span', class_='a-profile-name').text.strip()    

        # Amazon's format is "x.0 stars out of 5" where x = # of stars
        rating = html_review.find('span', class_='a-icon-alt').text.strip()[0]

        review_body = html_review.find('span', attrs={'data-hook': 'review-body'}).text.strip()

        reviews.append({'customer_name': name, 'rating': int(rating),'review': review_body})

    return reviews

In [326]:
__get_rated_reviews('https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=one_star&pageNumber=1')

[{'customer_name': 'Hayley', 'rating': 1, 'review': 'To soft'},
 {'customer_name': 'Rohit',
  'rating': 1,
  'review': 'I had bought a refurbished iPhone 14 that was unlocked and was having troubles setting up the eSIM (because iphone14 and up are eSIM only) through various trials and errors we had looked up the IEMI number up on T-Mobile data base and it had been reported as stolen through xfinity, now I have a phone that I can’t uses because of this, unbelievable.'},
 {'customer_name': 'Krystal',
  'rating': 1,
  'review': "This was the worst decision. I got the phone and the screen did not work. The screen is not apple and can tell it doesn't correctly fit. Even if I replaced it, T-Mobile said I would have the same problem. Don't buy from 3rd party. This was a scam."},
 {'customer_name': 'Zhangbaoquan',
  'rating': 1,
  'review': 'The screen often goes black and the phone freezes. This phone must have been severely dropped or soaked in water. It is unethical to sell such a phone.'},

In [327]:
star_page_suffix = {
    5: '/ref=cm_cr_unknown?filterByStar=five_star&pageNumber=',
    4: '/ref=cm_cr_unknown?filterByStar=four_star&pageNumber=',
    3: '/ref=cm_cr_unknown?filterByStar=three_star&pageNumber=',
    2: '/ref=cm_cr_unknown?filterByStar=two_star&pageNumber=',
    1: '/ref=cm_cr_unknown?filterByStar=one_star&pageNumber=',
}

AMAZON_REVIEW_URL = 'https://www.amazon.ca/product-reviews/'

In [330]:
def get_reviews(asin: str, num_reviews: int):
    assert num_reviews % 5 == 0

    base_url = AMAZON_REVIEW_URL + asin
    overall_reviews = []

    for star_num in range(1, 6):
        url = base_url + star_page_suffix[star_num]

        page_number = 1
        reviews = []
        reviews_per_star = int(num_reviews / 5)

        while len(reviews) <= reviews_per_star:
            page_url = url + str(page_number)
            print(page_url)

            # no reviews means we've exhausted all reviews
            page_reviews = __get_rated_reviews(page_url)

            if len(page_reviews) == 0:
                break

            reviews += page_reviews
            page_number += 1

        # shave off extra reviews coming from the last page
        reviews = reviews[:reviews_per_star]
        overall_reviews += reviews

    return overall_reviews

In [331]:
# num_reviews must be divisible by 10
get_reviews(asin = 'B0BN72FYFG', num_reviews = 10)

https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=one_star&pageNumber=1
https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=two_star&pageNumber=1
https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=three_star&pageNumber=1
https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=four_star&pageNumber=1
https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=five_star&pageNumber=1


[{'customer_name': 'Hayley', 'rating': 1, 'review': 'To soft'},
 {'customer_name': 'Rohit',
  'rating': 1,
  'review': 'I had bought a refurbished iPhone 14 that was unlocked and was having troubles setting up the eSIM (because iphone14 and up are eSIM only) through various trials and errors we had looked up the IEMI number up on T-Mobile data base and it had been reported as stolen through xfinity, now I have a phone that I can’t uses because of this, unbelievable.'},
 {'customer_name': 'Sb',
  'rating': 2,
  'review': 'Phone was good, great condition and all but it seems it was blacklisted (either someone didn’t pay it off or stole it, then it ended up on third party seller hands) so it was practically useless as a phone since I couldn’t connect it with carrier even with it being unlocked. It was huge pain, I was without a phone for almost a week until I needed up ordering a SIM card for my old phone which I’m so glad I didn’t trade in.The only reason I’m giving it two stars is becau

next:
- make it so that instead of num_reviews reviews per star, it uses equal amount per star (done)
- move this implementation into amazon.py (done)
- set upheadless chrome 